In [1]:
import random
from pyspark.sql import SparkSession
from src.extract import ingest_data
from src.transform import basic_preprocessing, deduplicate_df, df_join, groupby_expr, add_column
from config import (DATA_PATH, PAGEVIEWS_FILENAME, WIKI_NAMESPACE, MONGO_DB_NAME,
                    MONGO_COLLECTION_NAME, MONGO_URI, BATCH_SIZE,
                    POSTGRES_DB_CONFIG, POSTGRES_MAIN_TABLE_NAME, POSTGRES_ANALYTICS_TABLE_NAME)
from src.load_mongo import upload_to_mongo
from src.load_postgresql import load_to_postgres


In [2]:
# -- EXTRACT PHASE --
spark = SparkSession.builder.getOrCreate()
pageviews_data = ingest_data(spark, f"{DATA_PATH}/{PAGEVIEWS_FILENAME}", filetype="csv")
print(f"Pageviews data row count: {pageviews_data.count()}")
pageviews_data.printSchema()
# sample of pageviews data
pageviews_data.show(5, truncate=False)
print(f"Number of Spark partitions in pageviews data: {pageviews_data.rdd.getNumPartitions()}")

articles_data = ingest_data(spark, f"{DATA_PATH}/{WIKI_NAMESPACE}", filetype="jsonl")
print(f"Articles' data row count: {articles_data.count()}")
articles_data.printSchema()
# sample of articles data
articles_data.show(5, truncate=False)
print(f"Number of Spark partitions in articles data: {articles_data.rdd.getNumPartitions()}")

26/01/13 13:25:27 WARN Utils: Your hostname, Vishaks-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.31.211.170 instead (on interface en0)
26/01/13 13:25:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/13 13:25:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Pageviews data row count: 365700
root
 |-- rank: integer (nullable = true)
 |-- article: string (nullable = true)
 |-- views: string (nullable = true)
 |-- date: string (nullable = true)

+----+---------------------------------------------+-------+----------+
|rank|article                                      |views  |date      |
+----+---------------------------------------------+-------+----------+
|1   |List_of_stock_market_crashes_and_bear_markets|1876491|2016/01/01|
|2   |Natalie_Cole                                 |609871 |2016/01/01|
|3   |Star_Wars:_The_Force_Awakens                 |353499 |2016/01/01|
|4   |Bobby_Leach                                  |301290 |2016/01/01|
|5   |Wayne_Rogers                                 |231465 |2016/01/01|
+----+---------------------------------------------+-------+----------+
only showing top 5 rows

Number of Spark partitions in pageviews data: 4


Articles' data row count: 7405030
root
 |-- abstract: string (nullable = true)
 |-- additional_entities: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- aspects: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- identifier: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- date_created: string (nullable = true)
 |-- date_modified: string (nullable = true)
 |-- description: string (nullable = true)
 |-- event: struct (nullable = true)
 |    |-- date_created: string (nullable = true)
 |    |-- date_published: string (nullable = true)
 |    |-- fail_count: long (nullable = true)
 |    |-- fail_reason: string (nullable = true)
 |    |-- identifier: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- identifier: long (nullable = true)
 |-- image: struct (nullable = true)
 |    |-- content_url: string (nullable = true)
 |    |-- height: long (nullable = true)
 |  

In [3]:
# -- Transform Phase-1 --
# Preprocess pageviews data with creation of cleaned, standardised "article_name" column
pageviews_df = basic_preprocessing(
    df=pageviews_data,
    columns_to_select=["article", "views", "date", "rank"],
    column_to_standardise="article",
    new_column_name="article_name"
)
pageviews_df.select("article", "article_name").show(5, truncate=False)

# Preprocess articles data with creation of cleaned, standardised "article_name" column
articles_df = basic_preprocessing(
    df=articles_data,
    columns_to_select=["name", "description", "url", "abstract"],
    column_to_standardise="name",
    new_column_name="article_name"
)
articles_df.select("name", "article_name").show(5, truncate=False)

# Keep only unique records for each article name
print(f"Before dedup: {articles_df.count()}")
articles_df = deduplicate_df(df=articles_df, subset_columns=["article_name"])
print(f"After dedup: {articles_df.count()}")

# Join the 2 dfs on "article_name" column
df = df_join(
    left_df=pageviews_df,
    right_df=articles_df,
    join_columns=["article_name"],
    join_type="left"
)
print(f"Joined df Row count: {df.count()}")
df.printSchema()
df.show(5, truncate=True)

+---------------------------------------------+--------------------------------------+
|article                                      |article_name                          |
+---------------------------------------------+--------------------------------------+
|List_of_stock_market_crashes_and_bear_markets|listofstockmarketcrashesandbearmarkets|
|Natalie_Cole                                 |nataliecole                           |
|Star_Wars:_The_Force_Awakens                 |starwarstheforceawakens               |
|Bobby_Leach                                  |bobbyleach                            |
|Wayne_Rogers                                 |waynerogers                           |
+---------------------------------------------+--------------------------------------+
only showing top 5 rows

+----------------------------------+-------------------------------+
|name                              |article_name                   |
+----------------------------------+------------------

Before dedup: 7405030


26/01/12 18:45:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/01/12 18:45:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/01/12 18:45:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/01/12 18:45:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/01/12 18:45:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/01/12 18:45:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/01/12 18:45:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
26/01/12 18:45:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


After dedup: 6878306
Joined df Row count: 365698
root
 |-- article_name: string (nullable = true)
 |-- article: string (nullable = true)
 |-- views: string (nullable = true)
 |-- date: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- url: string (nullable = true)
 |-- abstract: string (nullable = true)



+--------------------+--------------------+------+----------+----+--------------------+--------------------+--------------------+--------------------+
|        article_name|             article| views|      date|rank|                name|         description|                 url|            abstract|
+--------------------+--------------------+------+----------+----+--------------------+--------------------+--------------------+--------------------+
|bettywhitefilmogr...|Betty_White_filmo...| 51355|2022/01/02|  94|Betty White filmo...|                NULL|https://en.wikipe...|Betty White was a...|
|       bumblebeefilm|    Bumblebee_(film)| 50136|2019/01/01|  85|    Bumblebee (film)|2018 film by Trav...|https://en.wikipe...|Bumblebee is a 20...|
|         nataliecole|        Natalie_Cole|609871|2016/01/01|   2|        Natalie Cole|American singer (...|https://en.wikipe...|Natalie Maria Col...|
|           cushjumbo|          Cush_Jumbo| 50914|2022/01/02|  96|          Cush Jumbo|British

In [4]:
# -- Transform Phase-2 --
print(f"Check number of unique articles present in pageviews data: "
      f"{df.select("article_name").distinct().count()}")
# pre-sort the data based on article_name and date
mongo_loading_df = df.orderBy("article_name", "date")

# This groups on article_name, converts pageviews data into lists and
# also conveniently, removes older article and name columns
list_collection_expr = [
    "COLLECT_LIST(date) AS date",
    "COLLECT_LIST(views) AS views",
    "COLLECT_LIST(rank) AS rank",
    "FIRST(description) AS description",
    "FIRST(url) AS url",
    "FIRST(abstract) AS abstract"
]
mongo_loading_df = groupby_expr(df=mongo_loading_df,
                                group_col="article_name",
                                expr_list=list_collection_expr)
print(f"Check number of articles present in flattened data: {mongo_loading_df.count()}")
mongo_loading_df.show(5, truncate=True)

Check number of unique articles present in pageviews data: 52570
Check number of articles present in flattened data: 52570


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        article_name|                date|               views|                rank|         description|                 url|            abstract|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   10cloverfieldlane|[2016/01/15, 2016...|[233689, 144755, ...|[7, 12, 53, 80, 9...|2016 film by Dan ...|https://en.wikipe...|10 Cloverfield La...|
|   10gigabitethernet|        [2017/02/14]|            [196047]|                 [8]|Standards for Eth...|https://en.wikipe...|10 Gigabit Ethern...|
|112gripesaboutthe...|        [2021/07/17]|             [41128]|                [62]|1945 handbook iss...|https://en.wikipe...|112 Gripes About ...|
|115thunitedstates...|        [2018/11/07]|             [75465]|                [49]|2017–2019 legisla...|

In [5]:
# -- Transform Phase-3 --
# simple transformation, adding one column 'popularity_bucket' based on views column
popularity_expr = """
CASE
    WHEN CAST(views AS BIGINT) < 10000 THEN 'low'
    WHEN CAST(views AS BIGINT) < 100000 THEN 'medium'
    ELSE 'high'
END
"""
pageviews_df = add_column(df=pageviews_df,
                          column_name='popularity_bucket',
                          expression=popularity_expr)
pageviews_df.show(5, truncate=False)

analytics_expr_list = [
    "AVG(CAST(views AS BIGINT)) AS avg_views",
    "MAX(CAST(views AS BIGINT)) AS max_views",
    "MIN(rank) AS best_rank",
    "COUNT(DISTINCT date) AS total_days_ranked"
]
analytics_df = groupby_expr(df=pageviews_df, group_col="article_name", expr_list=analytics_expr_list)
print(f"Total records in analytics df is: {analytics_df.count()}")
analytics_df.show(5, truncate=False)

+---------------------------------------------+-------+----------+----+--------------------------------------+-----------------+
|article                                      |views  |date      |rank|article_name                          |popularity_bucket|
+---------------------------------------------+-------+----------+----+--------------------------------------+-----------------+
|List_of_stock_market_crashes_and_bear_markets|1876491|2016/01/01|1   |listofstockmarketcrashesandbearmarkets|high             |
|Natalie_Cole                                 |609871 |2016/01/01|2   |nataliecole                           |high             |
|Star_Wars:_The_Force_Awakens                 |353499 |2016/01/01|3   |starwarstheforceawakens               |high             |
|Bobby_Leach                                  |301290 |2016/01/01|4   |bobbyleach                            |high             |
|Wayne_Rogers                                 |231465 |2016/01/01|5   |waynerogers               

+------------------+------------------+---------+---------+-----------------+
|article_name      |avg_views         |max_views|best_rank|total_days_ranked|
+------------------+------------------+---------+---------+-----------------+
|riyadmahrez       |50718.916666666664|86739    |22       |12               |
|tiffanytrump      |106484.36111111111|672091   |5        |72               |
|summerolympicgames|63277.37037037037 |109115   |26       |27               |
|theresamay        |87839.82089552238 |526311   |2        |67               |
|blackpanther      |47595.333333333336|55352    |41       |3                |
+------------------+------------------+---------+---------+-----------------+
only showing top 5 rows



In [6]:
# -- LOAD PHASE --
# MongoDB Loading
# Convert Spark rows to Python dictionaries for upload to mongoDB
# data_to_insert_mongo = mongo_loading_df.toPandas().to_dict('records')
# print(f"Sample insert document to MongoDB: "
#       f"{data_to_insert_mongo[random.randint(0, len(data_to_insert_mongo) - 1)]}")
# upload_to_mongo(data_to_insert_mongo, MONGO_DB_NAME, MONGO_COLLECTION_NAME, MONGO_URI, BATCH_SIZE)
#
# # PostgreSQL Loading
# postgres_records = (
#     pageviews_df
#     .select('article_name', 'views', 'popularity_bucket', 'date', 'rank')
#     .toPandas()
#     .itertuples(index=False, name=None)
# )
#
# columns = ["article_name", "views", "popularity_bucket", "date", "rank"]
# column_types = {
#     "article_name": "TEXT",
#     "views": "BIGINT",
#     "popularity_bucket": "TEXT",
#     "date": "DATE",
#     "rank": "INT"
# }
# load_to_postgres(
#     records=postgres_records,
#     columns=columns,
#     column_types=column_types,
#     db_config=POSTGRES_DB_CONFIG,
#     table_name=POSTGRES_MAIN_TABLE_NAME,
# )

# load_to_postgres(records=analytics_df.toPandas().itertuples(index=False, name=None),
#                  columns=["article_name", "avg_views", "max_views", "best_rank", "total_days_ranked"],
#                  column_types={
#                      "article_name": "TEXT",
#                      "avg_views": "FLOAT",
#                      "max_views": "FLOAT",
#                      "best_rank": "INT",
#                      "total_days_ranked": "INT"
#                  },
#                  db_config=POSTGRES_DB_CONFIG,
#                  table_name=POSTGRES_ANALYTICS_TABLE_NAME)

Created PostgreSQL table 'pageviews_analytics'
Insertion of records into PostgreSQL table 'pageviews_analytics' complete!


In [8]:
# Mongo Collection Index creation
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["text_etl_pipeline"]
collection = db["articles_data"]

collection.create_index(
    [("article_name", 1)],
    name="idx_article_name"
)

'idx_article_name'